![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_RoBERTa.ipynb)

# Import ONNX RoBERTa models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for RoBERTa from HuggingFace and they have to be in `Fill Mask` category. Meaning, you cannot use RoBERTa models trained/fine-tuned on a specific task such as token/sequence classification.

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers[onnx]==4.48.3 optimum onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 916.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [roberta-base](https://huggingface.co/roberta-base) model from HuggingFace as an example and load it as a `ORTModelForFeatureExtraction`, representing an ONNX model.
- In addition to the RoBERTa model, we also need to save the tokenizer. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import RobertaTokenizer
from optimum.onnxruntime import ORTModelForFeatureExtraction

MODEL_NAME = "roberta-base"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForFeatureExtraction.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

('onnx_models/roberta-base/tokenizer_config.json',
 'onnx_models/roberta-base/special_tokens_map.json',
 'onnx_models/roberta-base/vocab.json',
 'onnx_models/roberta-base/merges.txt',
 'onnx_models/roberta-base/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 486304
-rw-r--r-- 1 root root       644 Jun 16 03:22 config.json
-rw-r--r-- 1 root root    456318 Jun 16 03:22 merges.txt
-rw-r--r-- 1 root root 496500406 Jun 16 03:22 model.onnx
-rw-r--r-- 1 root root       958 Jun 16 03:22 special_tokens_map.json
-rw-r--r-- 1 root root      1222 Jun 16 03:22 tokenizer_config.json
-rw-r--r-- 1 root root    999355 Jun 16 03:22 vocab.json


- We need to convert `vocab.json` to `vocab.txt` by extracting and sorting the tokens (one per line), and make sure `merges.txt` is also present.
- Move both `vocab.txt` and `merges.tx`t into an `assets/` folder inside your export path, Spark NLP looks for tokenizer files there when loading ONNX models.

In [6]:
import json

with open(f'{EXPORT_PATH}/vocab.json') as f, open(f'{EXPORT_PATH}/vocab.txt', 'w') as out:
    out.writelines(f"{k}\n" for k in json.load(f).keys())

!mkdir -p {EXPORT_PATH}/assets
!mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/merges.txt {EXPORT_PATH}/assets/

In [7]:
!ls -l {EXPORT_PATH}/assets

total 848
-rw-r--r-- 1 root root 456318 Jun 16 03:22 merges.txt
-rw-r--r-- 1 root root 407065 Jun 16 03:26 vocab.txt


## Import and Save RoBERTa in Spark NLP

- **Install and set up Spark NLP in Google Colab**
  - This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [8]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 26.2 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [9]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `BertEmbeddings` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `BertEmbeddings` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- `setStorageRef` is very important. When you are training a task like NER or any Text Classification, we use this reference to bound the trained model to this specific embeddings so you won't load a different embeddings by mistake and see terrible results 😊
- It's up to you what you put in `setStorageRef` but it cannot be changed later on. We usually use the name of the model to be clear, but you can get creative if you want!
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [10]:
from sparknlp.annotator import RoBertaEmbeddings

roberta = RoBertaEmbeddings.loadSavedModel(f"{EXPORT_PATH}", spark)\
    .setInputCols(["document",'token'])\
    .setOutputCol("roberta")\
    .setCaseSensitive(True)\
    .setStorageRef('roberta-base')

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [11]:
roberta.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [12]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX RoBERTa model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [13]:
! ls -l {MODEL_NAME}_spark_nlp

total 484952
drwxr-xr-x 4 root root      4096 Jun 16 03:31 fields
drwxr-xr-x 2 root root      4096 Jun 16 03:31 metadata
-rw-r--r-- 1 root root 496576300 Jun 16 03:32 roberta_onnx


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBERTa model 😊

In [14]:
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

roberta = RoBertaEmbeddings.load(f"{MODEL_NAME}_spark_nlp") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("roberta")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    roberta
])

data = spark.createDataFrame([
    ["William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist."]
]).toDF("text")

result = pipeline.fit(data).transform(data)
result.selectExpr("explode(roberta.embeddings) as embedding").show()

+--------------------+
|           embedding|
+--------------------+
|[-0.0506, 0.15561...|
|[0.01447821, 0.16...|
|[-0.09041509, 0.1...|
|[-0.20549965, 0.0...|
|[-0.021994494, 0....|
|[-0.021145232, 0....|
|[-0.09210941, 0.0...|
|[-0.13212018, 0.0...|
|[-0.053157724, 8....|
|[0.011122618, 0.1...|
|[-0.04751993, 0.0...|
|[0.031325795, 0.3...|
|[-0.13260539, 0.0...|
|[-0.040670298, 0....|
|[0.04773803, -0.0...|
|[-0.023275442, 0....|
|[-0.07809764, 0.1...|
|[-0.10586174, 0.0...|
|[-0.07914694, 0.0...|
|[-0.18737869, 0.0...|
+--------------------+
only showing top 20 rows



That's it! You can now go wild and use hundreds of RoBERTa models from HuggingFace 🤗 in Spark NLP 🚀
